In [31]:
from tensorflow.keras import datasets, layers, models
import cvnn.initializers as initializers
from cvnn.activations import apply_activation
import numpy as np

(train_images, train_labels), (test_images, test_labels) = datasets.cifar10.load_data()

# Normalize pixel values to be between 0 and 1
train_images, test_images = train_images / 255.0, test_images / 255.0

In [48]:
class ComplexDense(layers.Layer):
    def __init__(self, units, activation=None, weight_initializer=None, bias_initializer=None, dtype=np.complex64):
        super(ComplexDense, self).__init__(dtype=dtype)
        if activation is None:
            activation = 'linear'
        self.activation = activation
        if weight_initializer is None:
            weight_initializer = initializers.GlorotUniform()
        self.weight_initializer = weight_initializer
        if bias_initializer is None:
            bias_initializer = initializers.Zeros()
        self.bias_initializer = bias_initializer
        self.units = units

    def build(self, input_shape):
        self.w = self.add_weight(
                                 initializer=self.weight_initializer,
                                 trainable=True,
                                 shape=[int(input_shape[-1]), self.units]
                                )
        self.b = self.add_weight(
            shape=(self.units,), initializer=self.bias_initializer, trainable=True
        )

    def call(self, inputs, training=None):
        out = tf.add(tf.matmul(inputs, self.w), self.b)
        y_out = apply_activation(self.activation, out)
        return y_out


In [49]:
model = models.Sequential()
model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(32, 32, 3)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.Flatten())
model.add(ComplexDense(64, activation='cart_relu', dtype=np.complex64))
model.add(ComplexDense(10, dtype=np.float32))

TypeError: in converted code:

    <ipython-input-48-d44630e7a540>:26 call  *
        out = tf.add(tf.matmul(inputs, self.w), self.b)
    C:\Users\NEGU93\.conda\envs\tf-2-gpu\lib\site-packages\tensorflow_core\python\util\dispatch.py:180 wrapper
        return target(*args, **kwargs)
    C:\Users\NEGU93\.conda\envs\tf-2-gpu\lib\site-packages\tensorflow_core\python\ops\math_ops.py:2798 matmul
        a, b, transpose_a=transpose_a, transpose_b=transpose_b, name=name)
    C:\Users\NEGU93\.conda\envs\tf-2-gpu\lib\site-packages\tensorflow_core\python\ops\gen_math_ops.py:5626 mat_mul
        name=name)
    C:\Users\NEGU93\.conda\envs\tf-2-gpu\lib\site-packages\tensorflow_core\python\framework\op_def_library.py:504 _apply_op_helper
        inferred_from[input_arg.type_attr]))

    TypeError: Input 'b' of 'MatMul' Op has type complex64 that does not match type float32 of argument 'a'.


In [45]:
model.summary()

Model: "sequential_15"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_42 (Conv2D)           (None, 30, 30, 32)        896       
_________________________________________________________________
max_pooling2d_26 (MaxPooling (None, 15, 15, 32)        0         
_________________________________________________________________
conv2d_43 (Conv2D)           (None, 13, 13, 64)        18496     
_________________________________________________________________
max_pooling2d_27 (MaxPooling (None, 6, 6, 64)          0         
_________________________________________________________________
conv2d_44 (Conv2D)           (None, 4, 4, 64)          36928     
_________________________________________________________________
flatten_12 (Flatten)         (None, 1024)              0         
_________________________________________________________________
complex_dense_8 (ComplexDens (None, 64)              

In [46]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

history = model.fit(train_images, train_labels, epochs=10, validation_data=(test_images, test_labels))

Train on 50000 samples, validate on 10000 samples
Epoch 1/10
50000/50000 [==============================] - 32s 641us/sample - loss: 1.5379 - accuracy: 0.4371 - val_loss: 1.2733 - val_accuracy: 0.5442
Epoch 2/10
50000/50000 [==============================] - 30s 597us/sample - loss: 1.1703 - accuracy: 0.5862 - val_loss: 1.0914 - val_accuracy: 0.6160
Epoch 3/10
50000/50000 [==============================] - 30s 608us/sample - loss: 1.0193 - accuracy: 0.6427 - val_loss: 1.0243 - val_accuracy: 0.6433
Epoch 4/10
50000/50000 [==============================] - 32s 648us/sample - loss: 0.9129 - accuracy: 0.6798 - val_loss: 1.0024 - val_accuracy: 0.658628 
Epoch 5/10
50000/50000 [==============================] - 33s 668us/sample - loss: 0.8394 - accuracy: 0.7047 - val_loss: 0.8921 - val_accuracy: 0.6910
Epoch 6/10
50000/50000 [==============================] - 34s 680us/sample - loss: 0.7758 - accuracy: 0.7313 - val_loss: 0.9111 - val_accuracy: 0.6845
Epoch 7/10
50000/50000 [=================